<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

# 2. Armado del ambiente

## 2.1 Armado del ambiente en CPU

No se requiere ejecuciones previas para la ejecución del programa

##2.2 Armado del ambiente en GPU

Se requiere la instalación del modulo de Cuda para Python

In [1]:
!pip install pycuda

#3. Desarrollo

##3.1 Desarrollo en CPU

In [2]:
#---------------------PARAMETROS-------------------------------
#@title 3.1.1 Parametros de ejecución {vertical-output: true}

cantidadElementos = 1000000#@param {type: "number"}

import numpy
import time

tiempoTotal = time.time() * 1000

vector = numpy.random.randint(1000, size = cantidadElementos)

longitud = len(vector)

print("Original:")
#print(vector)

#----------------------ALGORITMO----------------------------
tiempoBucle = time.time() *1000

n = len(vector) 
gap = n//2

while gap > 0: 

    for i in range(gap, n): 
        temp = vector[i] 
        j = i 
        while  j >= gap and vector[j-gap] >temp: 
            vector[j] = vector[j-gap] 
            j -= gap 

        vector[j] = temp 
    gap //= 2

#----------------------FINALIZACION----------------------------
tiempoBucle = (time.time() *1000) - tiempoBucle
print("Ordenado:")
#print(vector)

tiempoTotal = (time.time() *1000) - tiempoTotal
print("\n")
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

Original:
Ordenado:


Tiempo del bucle:  30113.286865234375 [ms.]
Tiempo del total:  30121.781005859375 [ms.]


##3.2 Desarrollo en GPU

In [12]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio =  1#@param {type: "number"}
orden_taylor = 0#@param {type: "number"}
punto_evalacion =  3#@param {type: "number"}
# --------------------------------------------

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import time
import numpy

tiempoTotal = (time.time() * 1000)

# Vectores en CPU.
coeficientes_cpu = numpy.random.randint(115, size = orden_polinomio + 1 )
coeficientes_cpu = coeficientes_cpu.astype(numpy.float32())

print("Coeficientes: ")
print(coeficientes_cpu)

terminos_taylor_cpu = numpy.zeros(orden_taylor + 1)
terminos_taylor_cpu = terminos_taylor_cpu.astype(numpy.float32())

resultado_cpu = numpy.zeros(orden_taylor + 1)
resultado_cpu = resultado_cpu.astype(numpy.float32())

# Vectores en GPU.
coeficientes_gpu = cuda.mem_alloc(coeficientes_cpu.nbytes)
terminos_taylor_gpu = cuda.mem_alloc(terminos_taylor_cpu.nbytes)

# Copio los datos
cuda.memcpy_htod(coeficientes_gpu, coeficientes_cpu)
cuda.memcpy_htod(terminos_taylor_gpu, terminos_taylor_cpu)

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_taylor( float punto, 
                               int long_pol, 
                               float *C, 
                               float *T){
  
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  
  float valor = 0;
  int idx_polinomio = 0;

  if(idx == 0){
    valor += C[idx_polinomio];
    
    float aux = 0;

    for(idx_polinomio = 1; idx_polinomio < long_pol + 1; idx_polinomio++){
      
    }
  }

  T[idx] = valor;

}
""")
print("\n")
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_taylor")

tiempo_gpu = (time.time() * 1000)

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (orden_taylor + 1 + dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel(numpy.float32(punto_evalacion),
       numpy.int32(orden_polinomio + 1),
       coeficientes_gpu, terminos_taylor_gpu, 
       block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1))

cuda.memcpy_dtoh(resultado_cpu, terminos_taylor_gpu)

tiempo_gpu = (time.time() * 1000) - tiempo_gpu

tiempo_total = (time.time() * 1000) - tiempoTotal
print(resultado_cpu)
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_total, "[ms]" )
print("Tiempo GPU: ", tiempo_gpu, "[ms]" )

Coeficientes: 
[35. 33. 77. 32.]


Thread x:  256 , Bloque x: 1
[35.]
Thread x:  256 , Bloque x: 1
Tiempo CPU:  6.3935546875 [ms]
Tiempo GPU:  1.085693359375 [ms]


#4. Tabla de pasos

#5. Conclusiones